# Searching

In [8]:
%classpath D:\\GoogleDrive\\wiki\\jupyter-notebooks\\Algorithms\\java

// 编译:: build.sh
// 清理: clean.sh

# 3.1 Symbol Tables

## 基于无序链表的顺序查找
* SequentialSearchST

# 3.2 Binary Search Trees

In [4]:

import java.util.*;

import adt.Queue;

/**
 * 二叉查找树.
 * 
 * 实现说明: 沿着根节点遍历左右子树, 执行递归操作.
 * 
 * 性质和实现:
 *
 * (1) 键
 * 一颗二叉树, 没有节点都含有一个Comparable的键及其值;
 * 每个节点的键大于其左子树中的任意节点的键, 小于右子树的任意节点的键.
 *
 * (2) 查找[插入]: 在以当前节点为根的子树中
 * 当前节点为null, 未命中[新创建节点].
 * 查找键小于当前节点键, 在左子树中查找[插入];
 * 查找键大于当前节点键, 在右子树中查找[插入];
 * 查找键等于当前节点键, 命中[更新值].
 *
 * (3) 删除
 * 待删除节点x
 * 左右子树节点有一个为null: 以左子节点为null为例, 将树中原指向x的引用指向x.right
 * 左右子树节点均不为null: 用T. Hibbard的方法使用x的后继节点替换x, 后继节点为x的右子树中的最小键节点.
 *
 * (4) 范围查找
 * 使用中序遍历, 保持遍历结果有序.
 */
// 约束: 每个节点的键大于其左子树中的任意节点的键, 小于右子树的任意节点的键.
public class BST<Key extends Comparable<Key>, Value> implements IOrderedST<Key, Value> {

    private class Node {

        private Key key; // 键
        private Value val; // 值
        private Node left, right; // 左右子树链接
        // 约束: size(x) = size(x.left) + size(x.right) + 1
        private int N = 0; // 以该节点为根的子树中的节点总数

        public Node(Key key, Value val, int N) {
            this.key = key;
            this.val = val;
            this.N = N;
        }

        @Override
        public String toString() {
            return "Node [key=" + key + ", val=" + val + ", left=" + left + ", right=" + right + ", N="
                    + N + "]";
        }
    }

    private Node root; // 二叉查找树的根节点

    @Override
    public void put(Key key, Value val) {
        if (key == null) {
            throw new IllegalArgumentException("null argument key!");
        }

        if (val == null) {
            this.delete(key);
            return;
        }

        root = this.put(root, key, val);
    }

    // 在以当前节点x为根的子树中插入键值对
    // key在子树中存在, 则更新其值; 否则在子树中创建新节点
    // x为null时, 返回新创建的节点, 否则返回当前节点x
    private Node put(Node x, Key key, Value val) {
        if (x == null) {
            return new Node(key, val, 1);
        }

        int cmp = key.compareTo(x.key);
        if (cmp < 0) {
            // 在左子树中插入
            x.left = this.put(x.left, key, val);
        } else if (cmp > 0) {
            // 在右子树中插入
            x.right = this.put(x.right, key, val);
        } else {
            x.val = val;
        }

        x.N = this.size(x.left) + this.size(x.right) + 1;
        return x;
    }

    @Override
    public Value get(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }

        return this.get(root, key);
    }

    // 在以节点x为根的树中查找key
    private Value get(Node x, Key key) {
        if (x == null) {
            return null;
        }

        int cmp = key.compareTo(x.key);
        if (cmp < 0) {
            // 在左子树中查找
            return this.get(x.left, key);
        } else if (cmp > 0) {
            // 在右子树中查找
            return this.get(x.right, key);
        } else {
            // 命中
            return x.val;
        }
    }

    @Override
    public void delete(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }
        if (this.isEmpty()) {
            throw new IllegalArgumentException("empty tree!");
        }

        root = this.delete(root, key);
    }

    /**
     * 在当前节点x为根的子树中删除键key对应的节点, 返回删除后的当前子树根节点x的左右子节点或后继节点.
     *
     * <pre>
     * 待删除节点x有左右子节点时, 使用T. Hibbard的后继节点替换待删除节点x方法:
     * (1) 将t指向待删除节点x
     * (2) 将x指向x的右子树中最小键节点
     * (3) 将x.right指向在t的右子树中删除最小键节点后的根节点
     * (4) 将x.left指向t.left, 完成替换.
     * </pre>
     *
     * @param x
     * @param key
     * @return
     */
    private Node delete(Node x, Key key) {
        if (x == null) {
            return null;
        }

        int cmp = key.compareTo(x.key);
        if (cmp < 0) {
            // 在左子树中删除
            x.left = this.delete(x.left, key);
        } else if (cmp > 0) {
            // 在右子树中删除
            x.right = this.delete(x.right, key);
        } else {
            // 删除当前节点
            if (x.left == null) {
                return x.right;
            } else if (x.right == null) {
                return x.left;
            } else {
                // 当前节点有左右子树
                Node t = x;
                x = this.min(x.right);
                x.right = this.deleteMin(t.right);
                x.left = t.left;
            }
        }
        x.N = this.size(x.left) + this.size(x.right) + 1;
        return x;
    }

    @Override
    public boolean contains(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }

        return this.get(key) != null;
    }

    @Override
    public boolean isEmpty() {
        return this.size() == 0;
    }

    @Override
    public int size() {
        return this.size(root);
    }

    // 以节点为根的子树中节点的数量
    private int size(Node x) {
        if (x == null) {
            return 0;
        } else {
            return x.N;
        }
    }

    @Override
    public Key min() {
        return this.min(root).key;
    }

    // 在以当前节点x为根的子树中查找有最小键的节点
    private Node min(Node x) {
        if (x.left == null) {
            // 左子树为null, 当前节点有最小键
            return x;
        } else {
            // 在左子树中查找
            return this.min(x.left);
        }
    }

    @Override
    public Key max() {
        return this.max(root).key;
    }

    /**
     * @see #min(Node)
     */
    private Node max(Node x) {
        if (x.right == null) {
            return x;
        } else {
            return this.max(x.right);
        }
    }

    @Override
    public Key floor(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }

        Node x = this.floor(root, key);
        if (x == null) {
            return null;
        } else {
            return x.key;
        }
    }

    // 在以当前节点x为根的子树中查找键<=key的节点
    // key=x.key: 命中;
    // key<x.key: 小于等于key的最大键一定在左子树中;
    // key>x.key: 右子树中存在键小于等于key的节点时, 小于等于key的最大键在右子树中; 否则当前节点是小于等于key的最大键节点
    private Node floor(Node x, Key key) {
        if (x == null) {
            return null;
        }

        int cmp = key.compareTo(x.key);
        if (cmp == 0) {
            // 命中
            return x;
        } else if (cmp < 0) {
            // 在左子树中查找
            return this.floor(x.left, key);
        } else {
            // 在右子树中查找
            Node t = this.floor(x.right, key);
            if (t != null) {
                // 存在键<=key的节点
                return t;
            } else {
                return x;
            }
        }
    }

    @Override
    public Key ceiling(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }

        Node x = this.ceiling(root, key);
        if (x == null) {
            return null;
        } else {
            return x.key;
        }
    }

    // 在以当前节点x为根的子树中查找键>=key的节点
    private Node ceiling(Node x, Key key) {
        if (x == null) {
            return null;
        }

        int cmp = key.compareTo(x.key);
        if (cmp == 0) {
            // 命中
            return x;
        } else if (cmp > 0) {
            // 在右子树中查找
            return this.ceiling(x.right, key);
        } else {
            // 在左子树中查找
            Node t = this.ceiling(x.left, key);
            if (t != null) {
                // 存在键>=key的节点
                return t;
            } else {
                return x;
            }
        }

    }

    @Override
    public int rank(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null argument!");
        }

        return this.rank(root, key);
    }

    // 在以当前节点x为根节点的子树中, 计算<key的键的数量
    private int rank(Node x, Key key) {
        if (x == null) {
            return 0;
        }

        int cmp = key.compareTo(x.key);
        if (cmp < 0) {
            // key小于当前节点的键: rank为key在左子树中的rank
            return this.rank(x.left, key);
        } else if (cmp > 0) {
            // key大于当前节点的键: rank为左子树节点数量+1(当前节点)+key在右子树中的rank
            return 1 + this.size(x.left) + this.rank(x.right, key);
        } else {
            // key等于当前节点的键: rank为左子树中节点数量
            return this.size(x.left);
        }
    }

    @Override
    public Key select(int k) {
        if (k <= 0 || k >= this.size()) {
            throw new IllegalArgumentException("invalid argument");
        }

        Node x = this.select(root, k);
        if (x == null) {
            return null;
        } else {
            return x.key;
        }
    }

    // 在以当前节点x为根的子树中查找排名为k的节点
    private Node select(Node x, int k) {
        if (x == null) {
            return null;
        }

        int t = this.size(x.left);// 左子树中节点数量

        if (t > k) {
            // 左子树中节点数大于k: 在左子树中查找
            return this.select(x.left, k);
        } else if (t < k) {
            // 左子树中节点数小于k: 在右子树中查找减去左子树和当前节点数量后的排名
            return this.select(x.right, k - (t + 1));
        } else {
            // 左子树中节点数等于k: 当前节点即为结果
            return x;
        }
    }

    @Override
    public void deleteMin() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty tree!");
        }

        root = this.deleteMin(root);
    }

    // 在以当前节点x为根节点的子树中删除最小键节点
    // 找到最小键节点时(即遇到null左链接), 返回该节点的右链接
    // 未找到最小键节点时, 在左子树中递归调用
    private Node deleteMin(Node x) {
        if (x.left == null) {
            // 找到最小键节点
            return x.right;
        }

        // 在左子树中删除
        x.left = this.deleteMin(x.left);
        x.N = this.size(x.left) + this.size(x.right) + 1; // 更新子树中节点数量
        return x;
    }

    @Override
    public void deleteMax() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty tree!");
        }

        root = this.deleteMax(root);
    }

    // 在以当前节点x为根节点的子树中删除最大键节点
    private Node deleteMax(Node x) {
        if (x.right == null) {
            return x.left;
        }

        x.right = this.deleteMax(x.right);
        x.N = this.size(x.left) + this.size(x.right) + 1;
        return x;
    }

    @Override
    public int size(Key low, Key high) {
        if (low == null) {
            throw new IllegalArgumentException("null argument low!");
        }
        if (high == null) {
            throw new IllegalArgumentException("null argument high!");
        }

        if (low.compareTo(high) > 0) {
            return 0;
        }

        if (this.contains(high)) {
            return this.rank(high) - this.rank(low) + 1;
        } else {
            return this.rank(high) - this.rank(low);
        }
    }

    @Override
    public Iterable<Key> keys() {
        return this.keys(this.min(), this.max());
    }

    @Override
    public Iterable<Key> keys(Key low, Key high) {
        if (low == null || high == null) {
            throw new IllegalArgumentException("all null arguments!");
        }
        if (low.compareTo(high) > 0) {
            throw new IllegalArgumentException("invalid argument!");
        }

        Queue<Key> q = new Queue<>();
        this.keys(root, q, low, high);
        return q;
    }

    // 在以当前节点x为根节点的子树中查找键在[low, high]范围内的节点, 加入队列q中
    // 使用中序遍历
    private void keys(Node x,
            Queue<Key> q, // OUT
            Key low, Key high) {
        if (x == null) {
            return;
        }

        // 使用中序遍历: 左子树->当前节点->右子树
        int cmpLow = low.compareTo(x.key);
        int cmpHigh = high.compareTo(x.key);
        if (cmpLow < 0) {
            // 左子树中可能有>low的键节点
            this.keys(x.left, q, low, high);
        }
        if (cmpLow <= 0 && cmpHigh >= 0) {
            // 当前节点键在范围内: 加入队列中
            q.enqueue(x.key);
        }
        if (cmpHigh > 0) {
            // 右子树中可能有<high的键节点
            this.keys(x.right, q, low, high);
        }
    }

    public void dump() {
        this.print(root);
    }

    public void dumpTree() {
        System.out.println(root);
    }

    private void print(Node x) {
        if (x == null) {
            return;
        }

        // 中序遍历
        this.print(x.left);
        System.out.println(x.key + "(" + x.val + ")");
        this.print(x.right);
    }

    public void dumpGraphviz() {
        List<Node> outNodes = new ArrayList();
        List<Edge> outEdges = new ArrayList();
        this.dumpGraphviz(root, outNodes, outEdges);

        StringBuilder sb = new StringBuilder();
        sb.append("digraph " + this.getClass().getSimpleName() + " {\n");
        sb.append("\t").append("size=100; ratio=0.8; splines=polyline\n");
        sb.append("\t").append(
                "node [shape = \"record\", fixedsize=true, fontsize=11, width=.3, height=.3]\n");
        sb.append("\t").append("edge [arrowsize=0.5, minlen=1]\n\n");

        for (Node x : outNodes) {
            sb.append("\t").append(x.key).append("[label=\"{<f1> " + x.key + "| {<f0> | <f2>}}\"]\n");
        }
        sb.append("\n");
        for (Edge e : outEdges) {
            if (Direction.LEFT.equals(e.direction)) {
                sb.append("\t").append(e.source.key + ":f0 -> " + e.target.key + ":f1\n");
            } else if (Direction.RIGHT.equals(e.direction)) {
                sb.append("\t").append(e.source.key + ":f2 -> " + e.target.key + ":f1\n");
            }
        }

        sb.append("}");

        System.out.println(sb.toString());
    }

    private enum Direction {
        LEFT, RIGHT
    }

    // 用于dumpGraphviz
    private class Edge {

        Node source;
        Node target;
        Direction direction;

        Edge(Node source, Node target, Direction direction) {
            this.source = source;
            this.target = target;
            this.direction = direction;
        }
    }

    private void dumpGraphviz(Node x, List<Node> outNodes, List<Edge> outEdges) {
        if (x == null) {
            return;
        }

        dumpGraphviz(x.left, outNodes, outEdges);

        outNodes.add(x);
        if (x.left != null) {
            outEdges.add(new Edge(x, x.left, Direction.LEFT));
        }
        if (x.right != null) {
            outEdges.add(new Edge(x, x.right, Direction.RIGHT));
        }

        dumpGraphviz(x.right, outNodes, outEdges);
    }

    public static void main(String[] args) {


    }
}


BST<String, Integer> bst = new BST<>();
int i = 0;
for (String key : "S E A R C H E X A M P L E".split("\\s")) {
    bst.put(key, i);
    i++;
}

bst.dump();
bst.dumpTree();
// bst.dumpGraphviz(); // https://stamm-wilbrandt.de/GraphvizFiddle/

// System.out.println(Joiner.on(" ").join(bst.keys()).toString());
for (String key : bst.keys()) {
    System.out.println(key + " " + bst.get(key));
}

A(8)
C(4)
E(12)
H(5)
L(11)
M(9)
P(10)
R(3)
S(0)
X(7)
Node [key=S, val=0, left=Node [key=E, val=12, left=Node [key=A, val=8, left=null, right=Node [key=C, val=4, left=null, right=null, N=1], N=2], right=Node [key=R, val=3, left=Node [key=H, val=5, left=null, right=Node [key=M, val=9, left=Node [key=L, val=11, left=null, right=null, N=1], right=Node [key=P, val=10, left=null, right=null, N=1], N=3], N=4], right=null, N=5], N=8], right=Node [key=X, val=7, left=null, right=null, N=1], N=10]
A 8
C 4
E 12
H 5
L 11
M 9
P 10
R 3
S 0
X 7


# 3.3 Balanced Search Trees

平衡查找树: 2-3树, 红黑树.

设计约束: 在维持树的完美平衡的前提下为新键腾出空间.

## 2-3树(概念)
空树或由2-节点/3-节点组成, 其中:
- 2-节点: 1个键值, 2个链接(左链接中键小于该节点, 右链接中的键大于该节点);
- 3-节点: 2个键值, 3个链接(左链接中键小于该节点, 右链接中的键大于该节点, 中链接位于该节点两个键值之间).

查找: 二叉查找树的查找算法的简单扩展.

插入: 依据未命中的查找结束于的节点类型
- (1) 2-节点

插入键值替换为3-节点;

- (2) 只有一个3-节点

创建临时的4-节点再分解为3个2-节点: 根节点含中键, 左链接节点中含较小值, 右链接节点中含较大值; 树高加1;

在根节点成为临时4-节点时按此处理;

- (3) 父节点为2-节点的3-节点

与(2)类似, 创建临时的4-节点再分解为3个2-节点, 将中键向上传播插入父2-节点中;

- (4) 父节点为3-节点的3-节点

与(2)类似, 创建临时的4-节点再分解为3个2-节点, 将中键向上传播插入父3-节点中, 父3-节点成为临时4-节点再分解, 依次向上处理;

## 2 红黑树 RedBlackBST

## 红黑树

In [6]:

import adt.Queue;
import searching.IOrderedST;

/**
 * 红黑树.
 *
 * 两种链接: 红链接将两个2-节点连接起来构成一个3-节点, 黑链接是2-3树中普通链接.
 *
 * 红黑树是含有红黑链接的二叉查找树, 且: (a) 红链接均为左链接; (b) 没有一个节点同时和两条红链接相连; (c) 完美黑色平衡的.
 *
 * 旋转操作: 操作中可能会出现红色右链接或者两条连续的红链接, 需要旋转修复. (1) 左旋转: 将一条红右链接转化为左链接; (2) 右旋转:
 * 将一条红左链接转化为右链接. 旋转操作需要保持: 有序性和完美平衡性.
 *
 * TODO: need more attention on red black BST, especially on delete operation!
 */
public class RedBlackBST<Key extends Comparable<Key>, Value> implements IOrderedST<Key, Value> {

    private static final boolean RED = true;
    private static final boolean BLACK = false;

    private class Node {

        private Key key; // 键
        private Value val; // 值
        private Node left, right; // 左右子树
        // 父节点指向该节点的链接的颜色, 即节点的颜色为指向该节点的链接的颜色; 空链接为黑色
        private boolean color;
        private int N; // 以该节点为根的子树中节点总数

        public Node(Key key, Value val, boolean color, int N) {
            this.key = key;
            this.val = val;
            this.color = color;
            this.N = N;
        }
    }

    private Node root; // 根节点

    public RedBlackBST() {
    }

    @Override
    public void put(Key key, Value val) {
        if (key == null) {
            throw new IllegalArgumentException("null key");
        }
        if (val == null) {
            this.delete(key);
            return;
        }

        root = this.put(root, key, val);
        root.color = BLACK; // 根节点总为黑色

    }

    private Node put(Node h, Key key, Value val) {
        if (h == null) {
            return new Node(key, val, RED, 1); // 和父节点用红链接相连
        }

        int cmp = key.compareTo(h.key);
        if (cmp < 0) {
            h.left = this.put(h.left, key, val);
        } else if (cmp > 0) {
            h.right = this.put(h.right, key, val);
        } else {
            h.val = val;
        }

        // 单个右红链接
        if (this.isRed(h.right) && !this.isRed(h.left)) {
            h = this.rotateLeft(h);
        }
        // 连续两条左红链接
        if (this.isRed(h.left) && this.isRed(h.left.left)) {
            h = this.rotateRight(h);
        }
        // 左右红链接
        if (this.isRed(h.left) && this.isRed(h.right)) {
            this.flipColor(h);
        }

        h.N = this.size(h.left) + this.size(h.right) + 1;
        return h;
    }

    // 子节点由红变黑, 父节点由黑变红
    private void flipColor(Node h) {
        h.color = !h.color;
        h.left.color = !h.left.color;
        h.right.color = !h.right.color;
    }

    @Override
    public Value get(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key");
        }

        return this.get(root, key);
    }

    private Value get(Node x, Key key) {
        while (x != null) {
            int cmp = key.compareTo(x.key);
            if (cmp < 0) {
                x = x.left;
            } else if (cmp > 0) {
                x = x.right;
            } else {
                return x.val;
            }
        }
        return null;
    }

    @Override
    public void delete(Key key) {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        if (!this.contains(key)) {
            return;
        }

        // 左右子链接均为黑色, 将根节点置为红色
        if (!this.isRed(root.left) && !this.isRed(root.right)) {
            root.color = RED;
        }

        root = this.delete(root, key);
        if (!this.isEmpty()) {
            root.color = BLACK;
        }
    }

    private Node delete(Node h, Key key) {
        if (key.compareTo(h.key) < 0) {

            if (!this.isRed(h.left) && !this.isRed(h.left.left)) {
                h = moveRedLeft(h);
            }
            h.left = this.delete(h.left, key);

        } else {

            if (this.isRed(h.left)) {
                h = this.rotateRight(h);
            }
            if (key.compareTo(h.key) == 0 && (h.right == null)) {
                return null;
            }
            if (!this.isRed(h.right) && !this.isRed(h.right.left)) {
                h = this.moveRedRight(h);
            }
            if (key.compareTo(h.key) == 0) {
                Node x = this.min(h.right);
                h.key = x.key;
                h.val = x.val;
                h.right = this.deleteMin(h.right);
            } else {
                h.right = this.delete(h.right, key);
            }
        }

        return this.balance(h);
    }

    @Override
    public boolean contains(Key key) {
        return this.get(key) != null;
    }

    @Override
    public boolean isEmpty() {
        return root == null;
    }

    @Override
    public int size() {
        return this.size(root);
    }

    @Override
    public Key min() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        return this.min(root).key;
    }

    // 以x为根的子树中最小键节点
    private Node min(Node x) {
        if (x.left == null) {
            // 直到左链接为空
            return x;
        } else {
            return this.min(x.left);
        }
    }

    @Override
    public Key max() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        return this.max(root).key;
    }

    private Node max(Node x) {
        if (x.right == null) {
            // 直到右链接为空
            return x;
        } else {
            return this.max(x.right);
        }
    }

    @Override
    public Key floor(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key");
        }
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        Node x = this.floor(root, key);
        if (x == null) {
            return null;
        } else {
            return x.key;
        }
    }

    private Node floor(Node x, Key key) {
        if (x == null) {
            return null;
        }
        int cmp = key.compareTo(x.key);
        if (cmp == 0) {
            return x;
        } else if (cmp < 0) {
            return this.floor(x.left, key);
        } else {
            Node t = this.floor(x.right, key);
            if (t != null) {
                return t;
            } else {
                return x;
            }
        }
    }

    @Override
    public Key ceiling(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key");
        }
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        Node x = this.ceiling(root, key);
        if (x == null) {
            return null;
        } else {
            return x.key;
        }
    }

    private Node ceiling(Node x, Key key) {
        if (x == null) {
            return null;
        }
        int cmp = key.compareTo(x.key);
        if (cmp == 0) {
            return x;
        } else if (cmp > 0) {
            return ceiling(x.right, key);
        } else {
            Node t = ceiling(x.left, key);
            if (t != null) {
                return t;
            } else {
                return x;
            }
        }
    }

    @Override
    public int rank(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key");
        }
        return this.rank(key, root);
    }

    private int rank(Key key, Node x) {
        if (x == null) {
            return 0;
        }
        int cmp = key.compareTo(x.key);
        if (cmp < 0) {
            return this.rank(key, x.left);
        } else if (cmp > 0) {
            return 1 + this.size(x.left) + this.rank(key, x.right);
        } else {
            return this.size(x.left);
        }
    }

    @Override
    public Key select(int k) {
        if (k <= 0 || k >= this.size()) {
            throw new IllegalArgumentException("invalid argument!");
        }

        Node x = this.select(root, k);
        return x.key;
    }

    private Node select(Node x, int k) {
        int t = this.size(x.left);
        if (t > k) {
            return this.select(x.left, k);
        } else if (t < k) {
            return this.select(x.right, k - t - 1);
        } else {
            return x;
        }
    }

    @Override
    public void deleteMin() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        // 左右子链接均为黑色, 将根节点置为红色
        if (!this.isRed(root.left) && !this.isRed(root.right)) {
            root.color = RED;
        }

        root = deleteMin(root);
        if (!this.isEmpty()) {
            root.color = BLACK;
        }
    }

    private Node deleteMin(Node h) {
        if (h.left == null) {
            return null;
        }

        if (!isRed(h.left) && !isRed(h.left.left)) {
            h = moveRedLeft(h);
        }

        h.left = deleteMin(h.left);
        return balance(h);
    }

    // PRE: h为红色, h.left, h.left.left为黑色
    // POST: h.left或者其子节点置为红色
    private Node moveRedLeft(Node h) {

        this.flipColor(h);
        if (this.isRed(h.right.left)) {
            h.right = this.rotateRight(h.right);
            h = this.rotateLeft(h);
            this.flipColor(h);
        }
        return h;
    }

    // PRE: h为红色, h.right, h.right.left为黑色
    // POST: h.right或者其子节点置为红色
    private Node moveRedRight(Node h) {
        this.flipColor(h);
        if (this.isRed(h.left.left)) {
            h = this.rotateRight(h);
            this.flipColor(h);
        }
        return h;
    }

    // 保持红黑树的性质
    private Node balance(Node h) {

        // 红右链接
        if (this.isRed(h.right)) {
            h = this.rotateLeft(h);
        }
        // 连续的红左链接
        if (this.isRed(h.left) && this.isRed(h.left.left)) {
            h = this.rotateRight(h);
        }
        // 左右子联接均为红色
        if (this.isRed(h.left) && this.isRed(h.right)) {
            this.flipColor(h);
        }

        h.N = size(h.left) + size(h.right) + 1;
        return h;
    }

    @Override
    public void deleteMax() {
        if (this.isEmpty()) {
            throw new IllegalStateException("empty!");
        }
        // 左右子链接均为黑色, 将根节点置为红色
        if (!this.isRed(root.left) && !this.isRed(root.right)) {
            root.color = RED;
        }

        root = this.deleteMax(root);
        if (!this.isEmpty()) {
            root.color = BLACK;
        }
    }

    private Node deleteMax(Node h) {
        if (this.isRed(h.left)) {
            h = this.rotateRight(h);
        }

        if (h.right == null) {
            return null;
        }

        if (!this.isRed(h.right) && !this.isRed(h.right.left)) {
            h = moveRedRight(h);
        }

        h.right = this.deleteMax(h.right);

        return this.balance(h);
    }

    @Override
    public int size(Key low, Key high) {

        if (low == null) {
            throw new IllegalArgumentException("null low!");
        }
        if (high == null) {
            throw new IllegalArgumentException("null high!");
        }

        if (low.compareTo(high) > 0) {
            return 0;
        }
        if (this.contains(high)) {
            return this.rank(high) - this.rank(low) + 1;
        } else {
            return this.rank(high) - this.rank(low);
        }
    }

    @Override
    public Iterable<Key> keys(Key low, Key high) {

        if (low == null) {
            throw new IllegalArgumentException("null low!");
        }
        if (high == null) {
            throw new IllegalArgumentException("null high!");
        }

        Queue<Key> queue = new Queue<>();
        this.keys(root, queue, low, high);
        return queue;
    }

    private void keys(Node x, Queue<Key> queue, Key lo, Key hi) {
        if (x == null) {
            return;
        }
        int cmplo = lo.compareTo(x.key);
        int cmphi = hi.compareTo(x.key);
        if (cmplo < 0) {
            this.keys(x.left, queue, lo, hi);
        }
        if (cmplo <= 0 && cmphi >= 0) {
            queue.enqueue(x.key);
        }
        if (cmphi > 0) {
            this.keys(x.right, queue, lo, hi);
        }
    }

    @Override
    public Iterable<Key> keys() {
        if (this.isEmpty()) {
            return new Queue<>();
        }
        return this.keys(this.min(), this.max());
    }

    // ======================================== 辅助方法
    // 是否为红链入节点
    private boolean isRed(Node x) {
        if (x == null) {
            return false;
        }
        return x.color == RED;
    }

    // 左旋h的右链接, 返回旋转后的根节点
    private Node rotateLeft(Node h) {
        Node x = h.right;
        h.right = x.left;
        x.left = h;
        x.color = h.color;
        h.color = RED;
        x.N = h.N;
        h.N = 1 + this.size(h.left) + this.size(h.right);
        return x;
    }

    private int size(Node x) {
        if (x == null) {
            return 0;
        }
        return x.N;
    }

    // 右旋h的左链接, 返回旋转后的根节点
    private Node rotateRight(Node h) {
        Node x = h.left;
        h.left = x.right;
        x.right = h;
        x.color = h.color;
        h.color = RED;
        x.N = h.N;
        h.N = 1 + this.size(h.left) + this.size(h.right);
        return x;
    }

    public static void main(String[] args) {
        RedBlackBST<String, Integer> rb = new RedBlackBST<>();
        int i = 0;
        for (String key : "S E A R C H E X A M P L E".split("\\s")) {
            rb.put(key, i);
            i++;
        }
        rb.dump();

        System.out.println();
        rb.delete("E");
        rb.delete("M");
        rb.dump();
    }

    public void dump() {
        this.print(root);
    }

    private void print(Node x) {
        if (x == null) {
            return;
        }

        // 中序遍历
        this.print(x.left);
        System.out.println(x.key + "(" + x.val + ")");
        this.print(x.right);
    }
}

RedBlackBST<String, Integer> rb = new RedBlackBST<>();
int i = 0;
for (String key : "S E A R C H E X A M P L E".split("\\s")) {
    rb.put(key, i);
    i++;
}
rb.dump();

System.out.println();
rb.delete("E");
rb.delete("M");
rb.dump();


A(8)
C(4)
E(12)
H(5)
L(11)
M(9)
P(10)
R(3)
S(0)
X(7)

A(8)
C(4)
H(5)
L(11)
P(10)
R(3)
S(0)
X(7)


# 3.4 Hash Tables

使用散列的查找算法:
- (1) 使用算术操作(散列函数)将键转化为数组的索引;
- (2) 处理碰撞冲突(多个键被散列到相同的索引值).

两种解决碰撞的方法:
- (1) 拉链法 SeparateChainingHashST
- (2) 线性探测法 LinearProbingHashST

## 拉链法

In [9]:

import adt.Queue;
import searching.*;

/**
 * 基于拉链法的散列表.
 * 
 * 将大小为M的数组中每个元素指向一条链表, 其中每个节点为散列值为该元素索引的键值对.
 * 
 * 选择足够大的M, 使得链表尽可能的短, 以保证高效的查找.
 */
public class SeparateChainingHashST<Key, Value> implements IST<Key, Value> {

  private static int MIN_M = 4; // 最小的散列表大小

  private int N; // 键值对总数
  private int M; // 散列表的大小

  private SequentialSearchST<Key, Value>[] st; // 链表对象的数组

  @SuppressWarnings("unchecked")
  public SeparateChainingHashST(int M) {
    this.M = M;
    st = (SequentialSearchST<Key, Value>[]) new SequentialSearchST[M];
    for (int i = 0; i < M; i++) {
      st[i] = new SequentialSearchST<Key, Value>();
    }
  }

  // 计算键的哈希值
  private int hash(Key key) {
    // 对散列表大小M取模
    return (key.hashCode() & 0x7fffffff) % M;
  }

  // 调整散列表的大小
  private void resize(int chains) {
    SeparateChainingHashST<Key, Value> temp = new SeparateChainingHashST<>(chains);

    for (int i = 0; i < M; i++) {
      for (Key key : st[i].keys()) {
        temp.put(key, st[i].get(key));
      }
    }

    this.M = temp.M;
    this.N = temp.N;
    this.st = temp.st;
  }

  @Override
  public void put(Key key, Value val) {
    if (key == null) {
      throw new IllegalArgumentException("null key!");
    }
    if (val == null) {
      this.delete(key);
      return;
    }

    if (N >= 10 * M) {
      this.resize(2 * M); // 调整散列表大小
    }

    if (!this.contains(key)) {
      N++;
    }
    st[this.hash(key)].put(key, val);
  }

  @Override
  public Value get(Key key) {
    if (key == null) {
      throw new IllegalArgumentException("null key!");
    }
    return st[this.hash(key)].get(key);
  }

  @Override
  public void delete(Key key) {
    if (key == null) {
      throw new IllegalArgumentException("null key!");
    }
    if (this.contains(key)) {
      N--;
    }
    st[this.hash(key)].delete(key);

    if (M > MIN_M && N <= 2 * M) {
      this.resize(M / 2); // 调整散列表的大小
    }
  }

  @Override
  public boolean contains(Key key) {
    if (key == null) {
      throw new IllegalArgumentException("null key!");
    }
    return st[this.hash(key)].contains(key);
  }

  @Override
  public boolean isEmpty() {
    return N == 0;
  }

  @Override
  public int size() {
    return N;
  }

  @Override
  public Iterable<Key> keys() {
    Queue<Key> queue = new Queue<>();
    for (int i = 0; i < M; i++) {
      for (Key key : st[i].keys()) {
        queue.enqueue(key);
      }
    }
    return queue;
  }

}


## 线性探测法

In [10]:

import adt.Queue;
import searching.*;

/**
 * 基于线性探测的散列表.
 *
 * 开放地址散列表: 使用数组中的空位解决碰撞冲突.
 *
 * 线性探测法 当碰撞发生时, 直接检查散列表中的下一个位置(索引值加1), 产生结果: (1) 命中, 该位置的键与被查找的键相同; (2) 未命中,
 * 该位置没有键; (3) 继续查找, 该位置的键与被查找的键不同.
 */
public class LinearProbingHashST<Key, Value> implements IST<Key, Value> {

    private int N; // 键值对的总数
    private int M = 16; // 线性探测表的大小

    // 并行数组
    private Key[] keys; // 键
    private Value[] vals; // 值

    @SuppressWarnings("unchecked")
    public LinearProbingHashST(int capacity) {
        this.M = capacity;
        this.keys = (Key[]) new Object[capacity];
        this.vals = (Value[]) new Object[capacity];
    }

    // 计算键的哈希值
    private int hash(Key key) {
        // 对线性探测表大小M取模
        return (key.hashCode() & 0x7fffffff) % M;
    }

    // 调整线性探测表的大小
    private void resize(int capacity) {
        LinearProbingHashST<Key, Value> temp = new LinearProbingHashST<>(capacity);
        for (int i = 0; i < M; i++) {
            if (keys[i] != null) {
                temp.put(keys[i], vals[i]);
            }
        }
        this.keys = temp.keys;
        this.vals = temp.vals;
        this.M = capacity;
    }

    @Override
    public void put(Key key, Value val) {
        if (key == null) {
            throw new IllegalArgumentException("null key!");
        }
        if (val == null) {
            this.delete(key);
            return;
        }

        if (N >= M / 2) {
            this.resize(M * 2); // 调整线性探测表的大小
        }

        int i;
        // keys[i]==null表示未命中, +1为继续查找
        for (i = this.hash(key); keys[i] != null; i = (i + 1) % M) {
            if (keys[i].equals(key)) {
                // 命中
                vals[i] = val;
                return;
            }
        }

        // 未命中
        keys[i] = key;
        vals[i] = val;
        N++;
    }

    @Override
    public Value get(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key!");
        }
        // keys[i]==null表示未命中, +1为继续查找
        for (int i = this.hash(key); keys[i] != null; i = (i + 1) % M) {
            if (keys[i].equals(key)) {
                // 命中
                return vals[i];
            }
        }

        // 未命中
        return null;
    }

    @Override
    public void delete(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key!");
        }
        if (!this.contains(key)) {
            return;
        }
        // 定位键所在位置, 平行数组置为null
        int i = this.hash(key);
        while (!key.equals(keys[i])) {
            i = (i + 1) % M;
        }
        keys[i] = null;
        vals[i] = null;

        // 后续的重放入
        i = (i + 1) % M;
        while (keys[i] != null) {
            Key redoKey = keys[i];
            Value redoVal = vals[i];
            keys[i] = null;
            vals[i] = null;
            N--;
            this.put(redoKey, redoVal);
            i = (i + 1) % M;
        }

        N--;

        if (N > 0 && N == M / 8) {
            this.resize(M / 2);
        }
    }

    @Override
    public boolean contains(Key key) {
        if (key == null) {
            throw new IllegalArgumentException("null key!");
        }
        return this.get(key) == null;
    }

    @Override
    public boolean isEmpty() {
        return N == 0;
    }

    @Override
    public int size() {
        return N;
    }

    @Override
    public Iterable<Key> keys() {
        Queue<Key> queue = new Queue<>();
        for (int i = 0; i < M; i++) {
            if (keys[i] != null) {
                queue.enqueue(keys[i]);
            }
        }
        return queue;
    }

}
